In [5]:
import requests
from pymongo import MongoClient

# MongoDB connection details
mongo_host = "localhost"
mongo_port = 27017
database_name = "apple"
collection_name = "intraday_r"

# Alpha Vantage API details
api_key = "your_alpha_vantage_api_key"
symbol = "AAPL"
interval = "5min"  # Example: 5-minute interval data
url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&apikey={api_key}"

def get_data(url):
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data from the API. Status code:", response.status_code)
        return None

def connect_to_mongodb():
    # Create a MongoDB client
    client = MongoClient(mongo_host, mongo_port)

    # Access the specified database and collection
    db = client[database_name]
    collection = db[collection_name]
    
    return collection

def store_individual_records(collection, data):
    # Extract the time series data
    time_series = data.get("Time Series (5min)", {})
    
    # Convert each record to a document and insert it into MongoDB
    records = []
    for timestamp, values in time_series.items():
        record = {
            "symbol": symbol,
            "timestamp": timestamp,
            "open": float(values["1. open"]),
            "high": float(values["2. high"]),
            "low": float(values["3. low"]),
            "close": float(values["4. close"]),
            "volume": int(values["5. volume"])
        }
        records.append(record)
    
    # Insert all records as separate documents
    if records:
        result = collection.insert_many(records)
        #print("Inserted document IDs:", result.inserted_ids)
    else:
        print("No records to store.")

if __name__ == "__main__":
    # Fetch data from the API
    data = get_data(url)
    
    if data:
        # Connect to MongoDB and get the collection
        collection = connect_to_mongodb()
        
        # Store individual records as separate documents in MongoDB
        store_individual_records(collection, data)
        
        print("Data successfully stored in MongoDB!")
    else:
        print("No data to store.")


Data successfully stored in MongoDB!
